In [103]:
import duckdb
from flask import Flask, request
from flask_cors import CORS
from threading import Lock
import time
app = Flask(__name__)
CORS(app)

con = duckdb.connect(database=':memory:', read_only=False)
con.execute("CREATE TABLE fires AS SELECT * FROM read_csv_auto('../fires_with_dropped_cols.csv')")
dbLock = Lock()

In [104]:
def jsonifyDuckData(data):
    return [{"x": d[0], "y": d[1]} for d in data]

def get_queryid_query(q):
    return "SELECT query_id FROM queries_list WHERE query='{}'".format(q)

def get_init_bw_lineage_groupby(qid):
    return """CREATE TABLE lineage_{} AS SELECT (s.index+r.range_start) AS input, s.value AS output
                    FROM HASH_GROUP_BY_{}_0_SINK AS s, SEQ_SCAN_{}_0_RANGE AS r
                    WHERE r.chunk_id=s.chunk_id""".format(qid, qid, qid)

def get_init_bw_lineage_perfect_groupby(qid):
    return """CREATE TABLE lineage_{} AS SELECT (sink.index+r.range_start) AS input, out.index AS output
                    FROM PERFECT_HASH_GROUP_BY_{}_0_OUT AS out, PERFECT_HASH_GROUP_BY_{}_0_SINK AS sink, SEQ_SCAN_{}_0_RANGE AS r
                    WHERE sink.value=out.value AND sink.chunk_id=r.chunk_id""".format(qid, qid, qid, qid)



In [105]:
def executeSQL(sqls):
    print(sqls)
    dbLock.acquire()
    try:
        res = {"res": {sql['name']: con.execute(sql["value"]).fetchall() for sql in sqls}}
    finally:
        dbLock.release()
        
    return res



In [106]:
start = time.time()
Q1 = "SELECT STAT_CAUSE_DESCR, COUNT(*) FROM fires GROUP BY STAT_CAUSE_DESCR"
Q2 = "SELECT STATE, COUNT(*) FROM fires GROUP BY STATE"
Q3 = "SELECT FIRE_YEAR, COUNT(*) FROM fires GROUP BY FIRE_YEAR"
q = []
q.append({"value":"PRAGMA trace_lineage='ON'", "name":"trace"})
q.append({"value":Q1, "name": "Q1"})
q.append({"value": Q2, "name": "Q2"})
q.append({"value":Q3, "name": "Q3"})
q.append({"value":"PRAGMA trace_lineage='OFF'", "name":"trace"})
executeSQL(q)

q1_id = con.execute(get_queryid_query(Q1)).fetchdf().iloc[0,0]
q2_id = con.execute(get_queryid_query(Q2)).fetchdf().iloc[0,0]
q3_id = con.execute(get_queryid_query(Q3)).fetchdf().iloc[0,0]
q = []

q.append({"value": get_init_bw_lineage_groupby(q1_id), "name": "lineage_q1"})
q.append({"value": get_init_bw_lineage_groupby(q2_id), "name": "lineage_q2"})
q.append({"value": get_init_bw_lineage_perfect_groupby(q3_id), "name": "lineage_q3"})
denormalized_lineage = """create table lineage as SELECT 
            fires.rowid as input, lineage_{}.output as output_{}, lineage_{}.output as output_{}, lineage_{}.output as output_{}
            FROM fires, lineage_{}, lineage_{}, lineage_{}
            WHERE fires.rowid=lineage_{}.input
                    and fires.rowid=lineage_{}.input
                    and fires.rowid=lineage_{}.input""".format(q1_id, q1_id, q2_id, q2_id, q3_id, q3_id,q1_id, q2_id, q3_id, q1_id, q2_id, q3_id)
q.append({"value": denormalized_lineage, "name": "denormalized_lineage"})

executeSQL(q)
end = time.time()
duration = (end - start) * 1000
print("total time: {}".format(duration))

[{'name': 'trace', 'value': "PRAGMA trace_lineage='ON'"}, {'name': 'Q1', 'value': 'SELECT STAT_CAUSE_DESCR, COUNT(*) FROM fires GROUP BY STAT_CAUSE_DESCR'}, {'name': 'Q2', 'value': 'SELECT STATE, COUNT(*) FROM fires GROUP BY STATE'}, {'name': 'Q3', 'value': 'SELECT FIRE_YEAR, COUNT(*) FROM fires GROUP BY FIRE_YEAR'}, {'name': 'trace', 'value': "PRAGMA trace_lineage='OFF'"}]
[{'name': 'lineage_q1', 'value': 'CREATE TABLE lineage_5 AS SELECT (s.index+r.range_start) AS input, s.value AS output\n                    FROM HASH_GROUP_BY_5_0_SINK AS s, SEQ_SCAN_5_0_RANGE AS r\n                    WHERE r.chunk_id=s.chunk_id'}, {'name': 'lineage_q2', 'value': 'CREATE TABLE lineage_7 AS SELECT (s.index+r.range_start) AS input, s.value AS output\n                    FROM HASH_GROUP_BY_7_0_SINK AS s, SEQ_SCAN_7_0_RANGE AS r\n                    WHERE r.chunk_id=s.chunk_id'}, {'name': 'lineage_q3', 'value': 'CREATE TABLE lineage_9 AS SELECT (sink.index+r.range_start) AS input, out.index AS output\n

In [107]:

selected_index = ['0', '1']
fw_lineage = "SELECT output_{}, output_{} from lineage where output_{} IN ({})".format(q1_id, q2_id, q3_id, ','.join(selected_index))
executeSQL([{"value": fw_lineage, "name": "fw_lineage"}])

[{'name': 'fw_lineage', 'value': 'SELECT output_5, output_7 from lineage where output_9 IN (0,1)'}]


{'res': {'fw_lineage': [(1, 7),
   (1, 7),
   (3, 7),
   (3, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (3, 7),
   (1, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (4, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (3, 7),
   (2, 7),
   (1, 7),
   (8, 7),
   (5, 7),
   (2, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (5, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (8, 7),
   (1, 7),
   (1, 7),
   (3, 7),
   (3, 7),
   (3, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (2, 7),
   (1, 7),
   (3, 7),
   (3, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (1, 7),
   (8, 7),
   (1, 7),
